In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# Define transforms
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])


In [4]:
# Load data
train_dataset = datasets.ImageFolder("dataset/train/", transform=transform)
val_dataset = datasets.ImageFolder("dataset/test/", transform=transform)

In [5]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

# Class labels
class_names = train_dataset.classes
print("Classes:", class_names)

Classes: ['Audi', 'Hyundai Creta', 'Mahindra Scorpio', 'Rolls Royce', 'Swift', 'Tata Safari', 'Toyota Innova']


Build the Model

In [6]:
import torchvision.models as models
import torch.nn as nn

# Load pretrained model
model = models.resnet18(pretrained=True)

# Freeze the base layers
for param in model.parameters():
    param.requires_grad = False

# Replace the final layer
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, len(class_names))  # Number of car brands

model = model.to(device)


C:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Dell\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)


In [8]:
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")


Epoch [1/10], Loss: 1.4146
Epoch [2/10], Loss: 0.9196
Epoch [3/10], Loss: 0.7381
Epoch [4/10], Loss: 0.6465
Epoch [5/10], Loss: 0.5819
Epoch [6/10], Loss: 0.5267
Epoch [7/10], Loss: 0.5000
Epoch [8/10], Loss: 0.4694
Epoch [9/10], Loss: 0.4542
Epoch [10/10], Loss: 0.4300


In [13]:
from sklearn.metrics import classification_report
import numpy as np

model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.numpy())

print(classification_report(all_labels, all_preds, target_names=class_names))


                  precision    recall  f1-score   support

            Audi       0.89      0.92      0.91       199
   Hyundai Creta       0.80      0.72      0.76        67
Mahindra Scorpio       1.00      0.88      0.94        75
     Rolls Royce       0.90      0.73      0.81        74
           Swift       0.87      0.88      0.87       102
     Tata Safari       0.87      0.94      0.90       106
   Toyota Innova       0.91      0.96      0.94       190

        accuracy                           0.89       813
       macro avg       0.89      0.86      0.87       813
    weighted avg       0.89      0.89      0.89       813



In [14]:
torch.save(model.state_dict(), "car_class.pth")

In [16]:
from PIL import Image

def predict_image(image_path, model, transform):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image_tensor)
        _, predicted = torch.max(output, 1)
        return class_names[predicted.item()]

# Example usage:
print(predict_image("dataset/test/Swift/122.jpg", model, transform))

Swift
